In [1]:
from datasets import load_from_disk

In [2]:
ds = load_from_disk("stack-code/mathematica")
ds = ds.shuffle()

In [3]:
i = 0 

In [28]:
text = ds[i]
i += 1
print(text["content"])

Notebook[{
Cell[" ", "SymbolColorBar",
 CellMargins->{{Inherited, Inherited}, {-5, 
   0}},ExpressionUUID->"260b93cd-d004-48b1-80d7-e759daceda3d"],

Cell[TextData[{
 ButtonBox["BlackHoleAnalysis",
  BaseStyle->{"Link", "LinkTrail"},
  ButtonData->"paclet:BlackHoleAnalysis/guide/BlackHoleAnalysis"],
 StyleBox[" > ", "LinkTrailSeparator"],
 ButtonBox["BlackHoleAnalysis`Labels",
  BaseStyle->{"Link", "LinkTrail"},
  ButtonData->"paclet:BlackHoleAnalysis/guide/BlackHoleAnalysis`Labels"],
 StyleBox[" > ", "LinkTrailSeparator"],
 ButtonBox["QrLabel",
  BaseStyle->{"Link", "LinkTrail"},
  ButtonData->"paclet:BlackHoleAnalysis/ref/QrLabel"]
}], "LinkTrail",ExpressionUUID->"08804df0-4298-45c4-a0c2-523cd0bcb625"],

Cell[CellGroupData[{

Cell["QrLabel", \
"ObjectName",ExpressionUUID->"7a705292-8cec-4aab-9c02-e15204282a4f"],

Cell[BoxData[GridBox[{
   {"", Cell[TextData[{
     Cell[BoxData[
      RowBox[{
       ButtonBox["QrLabel",
        BaseStyle->"Link",
        ButtonData->"paclet:BlackHoleA